In [60]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [61]:
import dataiku
from dataiku import pandasutils as pdu
import pandas as pd

In [62]:
client = dataiku.api_client()
#prj= client.get_project(dataiku.default_project_key())
#datasets = prj.list_datasets()

#client = DSSClient(host, apiKey)
dss_projects = ['BASKETBALLv2']
#dss_projects = client.list_project_keys()
print(dss_projects)



['BASKETBALLv2']


In [63]:
for ProjectKey in dss_projects:
    project = client.get_project(ProjectKey)
    datasets = project.list_datasets()
    print("Datasets in Project: ", ProjectKey)
    index = 0
    for dataset in datasets:
        # Quick access to main information in the dataset list item
        print("Name: %s" % dataset.name)
        print("Type: %s" % dataset.type)
        print("Connection: %s" % dataset.connection)
        print("Tags: %s" % dataset.tags) # Returns a list of strings

        # You can also use the list item as a dict of all available dataset information
        print("Raw params: %s" % dataset['params'])
        print("Raw schema: %s" % dataset['schema'])
        
        if (dataset.type == 'PostgreSQL'):
#        if dataset.type == 'Snowflake':


            dataset_to_update = project.get_dataset(dataset.name)
#             print("Dataset to update:")
#             print(dataset_to_update)
            
            # Get dataset settings
            settings = dataset_to_update.get_settings()
            # get raw dataset settings, which is editable
            settings_raw = settings.get_raw()
            print("\n")

            print("Project: ", ProjectKey, " Dataset: ", dataset.name, "params:")
            print(settings_raw['params'])
            print("Project", ProjectKey, "Dataset", dataset.name, "params schema:")
#            print(settings_raw['params']['schema'])
            print("<<<<<<<<<<SHUDEBUG0<<<<<<<<<<<")
#            print(settings_raw['params']['catalog'])
            print(settings_raw['params']['table'])
            print(">>>>>>>>>>>>SHUDEBUG1>>>>>>>>>>>")
            print(settings_raw['schema'])
            # pp.pprint(settings_raw['params'])
            if 'schema' in settings_raw['params']:
                print(">>>>>>>>>>>>SHUDEBUG2<<<<<<<<<<<")
                print(settings_raw['params']['schema'])
                
                
#             if 'schema' in settings_raw['params']:
#                 if settings_raw['params']['schema'] in ['AWB_CDW','AWB_CDL', 'AWB_ADHOC','AWB_DATAIKU_CDW', 'AWB_DATAIKU_CDL','AWB_DATAIKU_ADHOC']:
#                     new_values = workflow_Alteration_df[workflow_Alteration_df['Existing_Source'.strip()] == settings_raw['params']['table']]
#                     update_check = new_values.empty
#                     if not new_values.empty:
#                         print("Inside If", update_check, new_values,"*****", new_values.index[0])
#                         settings_raw['params']['catalog'] = new_values['GB_Database'.strip()][new_values.index[0]]
#                         settings_raw['params']['schema'] = new_values['GB_Schema'.strip()][new_values.index[0]]
#                         settings_raw['params']['table'] = new_values['GB_TBL_Name'.strip()][new_values.index[0]]
#                         settings.save()
#                         settings_raw["schema"] = {"columns":[]}
#                         settings.save()
#                         settings = dataset_to_update.autodetect_settings()
#                         settings.save()
#                         new_values = None
#                        index += 1            

Datasets in Project:  BASKETBALLv2
Name: Basketball_EXP_TM
Type: Filesystem
Connection: filesystem_managed
Tags: []
Raw params: {'connection': 'filesystem_managed', 'path': '${projectKey}/Basketball_EXP_TM', 'notReadyIfEmpty': False, 'filesSelectionRules': {'mode': 'ALL', 'excludeRules': [], 'includeRules': [], 'explicitFiles': []}}
Raw schema: {'columns': [{'name': 'Player', 'type': 'string'}, {'name': 'Year', 'type': 'bigint'}, {'name': 'has_TOT_sum', 'type': 'bigint'}, {'name': 'count', 'type': 'bigint'}, {'name': '1980_sum', 'type': 'bigint'}, {'name': 'Age_min', 'type': 'bigint'}, {'name': 'Pos', 'type': 'string', 'meaning': 'Text'}, {'name': 'Age', 'type': 'bigint'}, {'name': 'Tm', 'type': 'string'}, {'name': 'has_TOT', 'type': 'bigint'}, {'name': 'G', 'type': 'bigint'}, {'name': 'Injury', 'type': 'string'}, {'name': 'GS', 'type': 'bigint'}, {'name': 'MP', 'type': 'double'}, {'name': 'FG', 'type': 'double'}, {'name': 'FGA', 'type': 'double'}, {'name': 'FGP', 'type': 'double'}, {'

In [64]:
# index = 0
# for dataset in datasets:
#     #pp.pprint(dataset)
#     print(dataset)
#     if dataset.type == 'Snowflake':
#         dataset_to_update = prj.get_dataset(dataset.name)
#         # Get dataset settings
#         settings = dataset_to_update.get_settings()
#         # get raw dataset settings, which is editable
#         settings_raw = settings.get_raw()
#         # pp.pprint(settings_raw['params'])
#         if 'schema' in settings_raw['params']:
#             if settings_raw['params']['schema'] in ['AWB_CDW','AWB_CDL', 'AWB_ADHOC','AWB_DATAIKU_CDW', 'AWB_DATAIKU_CDL','AWB_DATAIKU_ADHOC']:
#                 new_values = workflow_Alteration_df[workflow_Alteration_df['Existing_Source'.strip()] == settings_raw['params']['table']]
#                 update_check = new_values.empty
#                 if not new_values.empty:
#                     print("Inside If", update_check, new_values,"*****", new_values.index[0])
#                     settings_raw['params']['catalog'] = new_values['GB_Database'.strip()][new_values.index[0]]
#                     settings_raw['params']['schema'] = new_values['GB_Schema'.strip()][new_values.index[0]]
#                     settings_raw['params']['table'] = new_values['GB_TBL_Name'.strip()][new_values.index[0]]
#                     settings.save()
#                     settings_raw["schema"] = {"columns":[]}
#                     settings.save()
#                     settings = dataset_to_update.autodetect_settings()
#                     settings.save()
#                     new_values = None
#                     index += 1